# Data Quality Reports

### Load data and extract categorical and continuous column headers

In [1]:
import pandas as pd

df = pd.read_csv('data/training_data.csv', index_col=0)
categorical_features = ['at_home', 'winning_at_half_time','won_match']
categorical_features = ['full_time_result', 'half_time_result']
continuous_features = [col for col in df.columns if col not in categorical_features]

### Continuous features

In [2]:
cont_df = df[continuous_features]
cont_df

,half_time_home_goals,half_time_away_goals,home_possession,away_possession,home_total_shots,away_total_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls_committed,away_fouls_committed,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
0,3,1,56.3,43.7,25,19,15,13,3,5,9,7,2,3,0,1
1,0,0,46.4,53.6,13,16,9,9,6,6,19,8,2,1,0,0
2,2,0,47.9,52.1,14,11,6,6,7,2,15,6,3,4,0,0
3,1,0,41.7,58.3,8,14,3,7,0,6,17,17,2,7,0,2
4,1,1,48.4,51.6,13,13,12,7,5,6,11,13,2,1,0,0
5,2,1,58.5,41.5,21,6,11,2,7,3,14,13,2,3,0,1
6,0,1,67.4,32.6,11,7,4,4,10,6,14,13,0,1,0,0
7,2,0,59.5,40.5,17,9,9,5,10,2,12,10,2,1,0,1
8,0,1,49.8,50.2,12,10,3,6,8,5,14,5,2,3,0,1
9,0,0,70.1,29.9,14,3,4,2,7,0,12,8,0,0,0,0


### Categorical features

In [3]:
cat_df = df[categorical_features]
cat_df

,full_time_result,half_time_result
0,H,H
1,H,D
2,H,H
3,H,H
4,H,D
5,D,H
6,A,A
7,H,H
8,D,A
9,D,D


### Function for building the continuous data quality report

In [4]:
import collections

def continuous_data_quality_report(df):
    data_cols = df.columns
    counts = []
    perc_missing = []
    cardinality = []
    mins = []
    first_quartiles = []
    means = []
    medians = []
    third_quartiles = []
    maxs = []
    std_devs = []

    for col in data_cols:
        counts.append(len(df[col]))
        perc_missing.append(df[col].isnull().sum() / counts[-1] * 100)
        cardinality.append(len(df[col].unique()))
        mins.append(df[col].min(skipna=True))
        first_quartiles.append(df[col].quantile(0.25))
        means.append(df[col].mean())
        medians.append(df[col].median())
        third_quartiles.append(df[col].quantile(0.75))
        maxs.append(df[col].max())
        std_devs.append(df[col].std())

    data = collections.OrderedDict((
        ('Feature', data_cols),
        ('Count', counts),
        ('% Missing', perc_missing),
        ('Cardinality', cardinality),
        ('Min', mins),
        ('1st Quartile', first_quartiles),
        ('Mean', means),
        ('Median', medians),
        ('3rd Quartile', third_quartiles),
        ('Max', maxs),
        ('Std Dev', std_devs),
    ))

    return pd.DataFrame(data)

### Function for building categorical data quality report

In [5]:
import collections


def categorical_data_quality_report(df):

    data_cols = df.columns
    counts = []
    perc_missing = []
    cardinality = []
    modes = []
    mode_counts = []
    mode_percs = []
    second_modes = []
    second_mode_counts = []
    second_mode_percs = []

    for col in data_cols:
        counts.append(len(df[col]))
        perc_missing.append(list(cat_df[col].values).count('') / counts[-1] * 100)
        cardinality.append(len(df[col].unique()))
        modes.append(df[col].value_counts().index[0])
        mode_counts.append(df[col].value_counts().iloc[0])
        mode_percs.append(mode_counts[-1] / counts[-1] * 100)
        if cardinality[-1] > 1:
            second_modes.append(df[col].value_counts().index[1])
            second_mode_counts.append(df[col].value_counts().iloc[1])
            second_mode_percs.append(second_mode_counts[-1] / counts[-1] * 100)
        else:
            second_modes.append('N/A')
            second_mode_counts.append('N/A')
            second_mode_percs.append('N/A')

    data = collections.OrderedDict((
        ('Feature', data_cols),
        ('Count', counts),
        ('% Missing', perc_missing),
        ('Cardinality', cardinality),
        ('Mode', modes),
        ('Mode Count', mode_counts),
        ('Mode %', mode_percs),
        ('2nd Mode', second_modes),
        ('2nd Mode Count', second_mode_counts),
        ('2nd Mode %', second_mode_percs),
    ))

    return pd.DataFrame(data)

In [6]:
continuous_data_quality_report(cont_df)

,Feature,Count,% Missing,Cardinality,Min,1st Quartile,Mean,Median,3rd Quartile,Max,Std Dev
0,half_time_home_goals,6873,0.0,7,0.0,0.0,0.690819,0.0,1.0,6.0,0.837841
1,half_time_away_goals,6873,0.0,6,0.0,0.0,0.518260,0.0,1.0,5.0,0.718332
2,home_possession,6873,0.0,546,15.5,44.0,51.413313,51.5,58.9,83.4,10.497247
3,away_possession,6873,0.0,546,16.6,41.1,48.586658,48.5,56.0,84.5,10.497281
4,home_total_shots,6873,0.0,40,1.0,10.0,13.989088,13.0,17.0,43.0,5.211423
5,away_total_shots,6873,0.0,33,0.0,8.0,11.218682,11.0,14.0,35.0,4.498647
6,home_shots_on_target,6873,0.0,21,0.0,3.0,5.049178,5.0,7.0,20.0,2.768087
7,away_shots_on_target,6873,0.0,20,0.0,2.0,4.027208,4.0,5.0,20.0,2.376823
8,home_corners,6873,0.0,22,0.0,4.0,5.806344,5.0,8.0,22.0,3.001340
9,away_corners,6873,0.0,20,0.0,3.0,4.497599,4.0,6.0,19.0,2.573998


In [7]:
categorical_data_quality_report(cat_df)

,Feature,Count,% Missing,Cardinality,Mode,Mode Count,Mode %,2nd Mode,2nd Mode Count,2nd Mode %
0,full_time_result,6873,0.0,3,H,3110,45.249527,A,2022,29.419467
1,half_time_result,6873,0.0,3,D,2874,41.815801,H,2387,34.730103
